Lots of imports do not know what is needed and what is not needed.

if there is time it is possible to filter

In [1]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

In [2]:
!pip install -q pyspark
!pip install -U -q PyDrive
!apt-get update
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [3]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [4]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

Saves the dictionary to a file called "id_title_dict.pickle"

The file must be created before running

In [5]:

parquetFile_pd = pd.read_csv("small_corpus.csv")
parquetFile = spark.createDataFrame(parquetFile_pd)
doc_title_pairs = parquetFile.select("title","id")

id_title_dict = {}
for i in doc_title_pairs.collect():
  id_title_dict[i.id] = i.title

with open('id_title_dict.pickle', 'wb') as f:
  pickle.dump(id_title_dict, f)



Example of reading the file

The dictionary will be in format id:title

In [6]:
with open('id_title_dict.pickle', 'rb') as f:
  wid2title = pickle.loads(f.read())

print(wid2title)

{4055486: 'Time Walker', 4080953: 'Nazi racial theories', 4093674: 'Siopao', 4105482: 'Plain vanilla', 4108839: 'NBA on TNT', 4141563: 'Predictive analytics', 4146044: 'Shrimp toast', 4148655: 'Jeremy Renner', 4169317: 'Goat tying', 4189740: 'Plant defense against herbivory', 4193788: 'Making Money', 4201044: 'Masks (Star Trek: The Next Generation)', 4206029: 'Familial hemiplegic migraine', 4225907: 'Docstring', 4242777: 'Raja Yoga (book)', 4250574: 'Chocolate (Snow Patrol song)', 4275382: 'Pasta e fagioli', 4301719: 'Mask (Bauhaus album)', 4338696: 'History of Google', 4344526: 'Mango pudding', 4355398: 'Etsy', 4361873: 'Rake (poker)', 4390795: 'Vavilovian mimicry', 4427231: 'Fairy Tale (Michael Wong album)', 4432842: 'Chocolate Puma', 4451883: 'Anthony Mackie', 4456418: 'Sprite Remix', 4463369: 'List of Darkwing Duck episodes', 4465875: 'Google Finance', 4465969: 'List of tallest buildings and structures in the Paris region', 4468939: 'Orecchiette', 4481567: 'Nightmare Cafe', 4512778